In [1]:
# Instalar bibliotecas
!pip install pandas google-cloud-storage mysql-connector-python
!pip install --upgrade google-cloud-storage

# Importar bibliotecas
import pandas as pd
from google.cloud import storage
import mysql.connector
from mysql.connector import Error
import os

# Autenticar a conta do Google Cloud
from google.colab import auth
auth.authenticate_user()

# Configurar o cliente do Google Cloud Storage
client = storage.Client()
bucket_name = 'projeto01bucketcolabsql'

# Caminhos dos arquivos
csv_file_path1 = '/content/bal.patrim.petrobras_31.03.09.csv'
csv_file_path2 = '/content/dem.resultado.petrobras.csv'
excel_file_path1 = '/content/balanco_petro.xlsx'
excel_file_path2 = '/content/balanco_petro(editada).xlsx'

# Função para fazer upload de um arquivo para o bucket GCS
def upload_to_gcs(bucket_name, file_path):
    bucket = client.get_bucket(bucket_name)
    file_name = os.path.basename(file_path)
    blob = bucket.blob(f'dados_brutos/{file_name}')
    blob.upload_from_filename(file_path)
    print(f'Arquivo {file_path} carregado para {bucket_name}/dados_brutos/{file_name}')

# Função para ler um arquivo CSV do GCS
def read_csv_from_gcs(bucket_name, file_name):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)

    # Defina o caminho de destino e crie o diretório se ele não existir
    local_file_path = '/content/' + file_name
    local_dir = os.path.dirname(local_file_path)
    os.makedirs(local_dir, exist_ok=True)

    # Faça o download do arquivo
    blob.download_to_filename(local_file_path)

    # Leia o arquivo CSV para um DataFrame
    df = pd.read_csv(local_file_path)
    return df

# Função para ler um arquivo Excel do GCS
def read_excel_from_gcs(bucket_name, file_name):
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)

    # Defina o caminho de destino e crie o diretório se ele não existir
    local_file_path = '/content/' + file_name
    local_dir = os.path.dirname(local_file_path)
    os.makedirs(local_dir, exist_ok=True)

    # Faça o download do arquivo
    blob.download_to_filename(local_file_path)

    # Leia o arquivo Excel para um DataFrame
    df = pd.read_excel(local_file_path, engine='openpyxl')
    return df

# Fazer upload dos arquivos CSV e Excel para o bucket
upload_to_gcs(bucket_name, csv_file_path1)
upload_to_gcs(bucket_name, csv_file_path2)
upload_to_gcs(bucket_name, excel_file_path1)
upload_to_gcs(bucket_name, excel_file_path2)

# Ler os arquivos CSV e Excel do GCS
df_csv1 = read_csv_from_gcs(bucket_name, 'dados_brutos/bal.patrim.petrobras_31.03.09.csv')
df_csv2 = read_csv_from_gcs(bucket_name, 'dados_brutos/dem.resultado.petrobras.csv')
df_excel1 = read_excel_from_gcs(bucket_name, 'dados_brutos/balanco_petro.xlsx')
df_excel2 = read_excel_from_gcs(bucket_name, 'dados_brutos/balanco_petro(editada).xlsx')

# Mostrar os primeiros registros dos DataFrames
print("Conteúdo do CSV 1:")
display(df_csv1.head())

print("\nConteúdo do CSV 2:")
display(df_csv2.head())

print("\nConteúdo do Excel 1:")
display(df_excel1.head())

print("\nConteúdo do Excel 2:")
display(df_excel2.head())

Arquivo /content/bal.patrim.petrobras_31.03.09.csv carregado para projeto01bucketcolabsql/dados_brutos/bal.patrim.petrobras_31.03.09.csv
Arquivo /content/dem.resultado.petrobras.csv carregado para projeto01bucketcolabsql/dados_brutos/dem.resultado.petrobras.csv
Arquivo /content/balanco_petro.xlsx carregado para projeto01bucketcolabsql/dados_brutos/balanco_petro.xlsx
Arquivo /content/balanco_petro(editada).xlsx carregado para projeto01bucketcolabsql/dados_brutos/balanco_petro(editada).xlsx
Conteúdo do CSV 1:


,Data,Ativo Total,Ativo Circulante,Caixa e Equivalentes de Caixa,Aplicações Financeiras,Contas a Receber,Estoques,Ativos Biológicos,Tributos a Recuperar,Despesas Antecipadas,...,Patrimônio Líquido,Capital Social Realizado,Reservas de Capital,Reservas de Reavaliação,Reservas de Lucros,Lucros/Prejuízos Acumulados,Ajustes de Avaliação Patrimonial,Ajustes Acumulados de Conversão,Outros Resultados Abrangentes,Adiantamento para Futuro Aumento Capital.1
0,30/06/2009,305.265.345,57.621.537,10.072.162,0,14.555.268,19.674.548,0,0,0,...,149.847.966,78.966.686,514.857,9.920,58.865.377,10.918.406,572.716,0,0,0
1,30/09/2009,333.789.528,75.719.057,30.088.286,0,13.643.311,20.635.085,0,0,0,...,155.391.689,78.966.686,514.857,9.718,58.866.672,16.466.363,567.399,0,0,0
2,31/12/2009,345.607.274,76.674.015,28.795.714,0,13.984.270,21.424.652,0,0,0,...,159.464.587,78.966.686,514.857,350.000,79.521.014,0,461.687,0,0,0
3,31/03/2010,365.998.080,74.459.103,26.951.326,0,16.200.355,20.030.609,0,0,0,...,170.299.081,78.966.686,514.857,0.000,84.879.696,5.971.459,-33.617,0,0,0
4,30/06/2010,382.029.857,71.980.237,24.209.867,0,15.961.582,19.680.465,0,0,0,...,176.974.135,85.108.548,0,0.000,79.252.693,12.494.329,118.554,0,0,0



Conteúdo do CSV 2:


,Data,Receita Bruta de Vendas e/ou Serviços,Deduções da Receita Bruta,Receita Líquida de Vendas e/ou Serviços,Custo de Bens e/ou Serviços Vendidos,Resultado Bruto,Despesas Com Vendas,Despesas Gerais e Administrativas,Perdas pela Não Recuperabilidade de Ativos,Outras Receitas Operacionais,...,Resultado Não Operacional,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período
0,30/06/2009,55.891.489,-11.287.491,44.603.994,-24.613.196,19.990.798,-1.745.850,-1.834.082,NaN,0,...,0.0,0.0,0.0,11.807.475,-3.852.421,1.656.294,0,0.0,-1.876.852,7.734.496
1,30/09/2009,60.264.415,-12.386.796,47.877.620,-29.015.413,18.862.207,-1.756.580,-1.965.524,NaN,0,...,0.0,0.0,0.0,11.263.481,-2.354.096,-679.049,0,0.0,-927.768,7.302.568
2,31/12/2009,60.772.676,-13.139.804,47.632.871,-29.628.371,18.004.513,-1.785.407,-1.857.617,NaN,0,...,0.0,0.0,0.0,11.135.240,-575.861,-1.329.499,-1.495.323,0.0,394.315,8.128.872
3,31/03/2010,63.323.783,-12.911.715,50.412.073,-31.101.667,19.310.399,-2.072.394,-1.829.000,NaN,0,...,0.0,0.0,0.0,10.736.777,-3.386.217,446.287,0,0.0,-70.573,7.726.274
4,30/06/2010,66.884.448,-13.253.071,53.631.382,-34.244.725,19.386.655,-2.276.030,-1.897.404,NaN,0,...,0.0,0.0,0.0,11.441.831,-1.563.613,-1.541.420,0,0.0,-41.808,8.294.990



Conteúdo do Excel 1:


,"XLSWrite 1.34 Copyright(c) 1999,2000 Axolot Data",Balanço Patrimonial - PETROBRAS,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60
0,31/03/2009,30/06/2009,30/09/2009,31/12/2009,31/03/2010,30/06/2010,30/09/2010,31/12/2010,31/03/2011,30/06/2011,...,31/12/2021,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024
1,Ativo Total,305265344.512,333789528.064,345607274.496,365998080,382029856.768,507697037.312,519970029.568,544945274.88,554583457.792,...,972950994.944,998662012.928,1004753977.344,947574013.952,976708960.256,978576998.4,990459002.88,1025495990.272,1050887979.008,1067292033.024
2,Ativo Circulante,57621536.768,75719057.408,76674015.232,74459103.232,71980236.8,111415033.856,106685161.472,120036376.576,119493476.352,...,168247001.088,199511998.464,224784990.208,159327010.816,163051995.136,157193994.24,136816001.024,147311001.6,157079011.328,165964005.376
3,Caixa e Equivalentes de Caixa,10072162.304,30088286.208,28795713.536,26951325.696,24209866.752,47291932.672,30323259.392,43344818.176,34672615.424,...,58410000.384,81601003.52,85310996.48,23650000.896,41722998.784,52276998.144,49882001.408,60642000.896,61612998.656,57689001.984
4,Aplicações Financeiras,0,0,0,0,0,0,26017296.384,20015771.648,24969263.104,...,3630000.128,5967000.064,14956999.68,13038000.128,14469999.616,14629000.192,11102999.552,6504999.936,13649999.872,24071999.488



Conteúdo do Excel 2:


,Data,Receita Bruta de Vendas e/ou Serviços,Deduções da Receita Bruta,Receita Líquida de Vendas e/ou Serviços,Custo de Bens e/ou Serviços Vendidos,Resultado Bruto,Despesas Com Vendas,Despesas Gerais e Administrativas,Perdas pela Não Recuperabilidade de Ativos,Outras Receitas Operacionais,...,Resultado Não Operacional,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período
0,30/06/2009,5.589149e+07,-1.128749e+07,4.460399e+07,-2.461320e+07,1.999080e+07,-1745849.984,-1834082.048,NaN,0,...,0.0,0.0,0.0,1.180747e+07,-3852421.120,1656294.016,0.000,0.0,-1876851.968,7734496.256
1,30/09/2009,6.026442e+07,-1.238680e+07,4.787762e+07,-2.901541e+07,1.886221e+07,-1756579.968,-1965523.968,NaN,0,...,0.0,0.0,0.0,1.126348e+07,-2354096.128,-679049.024,0.000,0.0,-927768.000,7302567.936
2,31/12/2009,6.077268e+07,-1.313980e+07,4.763287e+07,-2.962837e+07,1.800451e+07,-1785407.104,-1857617.024,NaN,0,...,0.0,0.0,0.0,1.113524e+07,-575860.736,-1329499.008,-1495323.008,0.0,394314.944,8128872.448
3,31/03/2010,6.332378e+07,-1.291172e+07,5.041207e+07,-3.110167e+07,1.931040e+07,-2072393.984,-1828999.936,NaN,0,...,0.0,0.0,0.0,1.073678e+07,-3386216.960,446287.008,0.000,0.0,-70573.000,7726274.048
4,30/06/2010,6.688445e+07,-1.325307e+07,5.363138e+07,-3.424472e+07,1.938665e+07,-2276029.952,-1897404.032,NaN,0,...,0.0,0.0,0.0,1.144183e+07,-1563613.056,-1541420.032,0.000,0.0,-41808.000,8294989.824


In [2]:

# Ler os arquivos CSV do GCS e transformá-los em DataFrames
df_dem_resultado = read_csv_from_gcs(bucket_name, 'dados_brutos/dem.resultado.petrobras.csv')
# Mostrar os primeiros registros dos DataFrames
print("\nConteúdo do df_dem_resultado:")
display(df_dem_resultado.head())


Conteúdo do df_dem_resultado:


,Data,Receita Bruta de Vendas e/ou Serviços,Deduções da Receita Bruta,Receita Líquida de Vendas e/ou Serviços,Custo de Bens e/ou Serviços Vendidos,Resultado Bruto,Despesas Com Vendas,Despesas Gerais e Administrativas,Perdas pela Não Recuperabilidade de Ativos,Outras Receitas Operacionais,...,Resultado Não Operacional,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período
0,30/06/2009,55.891.489,-11.287.491,44.603.994,-24.613.196,19.990.798,-1.745.850,-1.834.082,NaN,0,...,0.0,0.0,0.0,11.807.475,-3.852.421,1.656.294,0,0.0,-1.876.852,7.734.496
1,30/09/2009,60.264.415,-12.386.796,47.877.620,-29.015.413,18.862.207,-1.756.580,-1.965.524,NaN,0,...,0.0,0.0,0.0,11.263.481,-2.354.096,-679.049,0,0.0,-927.768,7.302.568
2,31/12/2009,60.772.676,-13.139.804,47.632.871,-29.628.371,18.004.513,-1.785.407,-1.857.617,NaN,0,...,0.0,0.0,0.0,11.135.240,-575.861,-1.329.499,-1.495.323,0.0,394.315,8.128.872
3,31/03/2010,63.323.783,-12.911.715,50.412.073,-31.101.667,19.310.399,-2.072.394,-1.829.000,NaN,0,...,0.0,0.0,0.0,10.736.777,-3.386.217,446.287,0,0.0,-70.573,7.726.274
4,30/06/2010,66.884.448,-13.253.071,53.631.382,-34.244.725,19.386.655,-2.276.030,-1.897.404,NaN,0,...,0.0,0.0,0.0,11.441.831,-1.563.613,-1.541.420,0,0.0,-41.808,8.294.990


In [3]:
# Exibir as primeiras 5 linhas dos DataFrames
display(df_dem_resultado.head())

# Exibir informações detalhadas do DataFrame
display(df_dem_resultado.info())

,Data,Receita Bruta de Vendas e/ou Serviços,Deduções da Receita Bruta,Receita Líquida de Vendas e/ou Serviços,Custo de Bens e/ou Serviços Vendidos,Resultado Bruto,Despesas Com Vendas,Despesas Gerais e Administrativas,Perdas pela Não Recuperabilidade de Ativos,Outras Receitas Operacionais,...,Resultado Não Operacional,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período
0,30/06/2009,55.891.489,-11.287.491,44.603.994,-24.613.196,19.990.798,-1.745.850,-1.834.082,NaN,0,...,0.0,0.0,0.0,11.807.475,-3.852.421,1.656.294,0,0.0,-1.876.852,7.734.496
1,30/09/2009,60.264.415,-12.386.796,47.877.620,-29.015.413,18.862.207,-1.756.580,-1.965.524,NaN,0,...,0.0,0.0,0.0,11.263.481,-2.354.096,-679.049,0,0.0,-927.768,7.302.568
2,31/12/2009,60.772.676,-13.139.804,47.632.871,-29.628.371,18.004.513,-1.785.407,-1.857.617,NaN,0,...,0.0,0.0,0.0,11.135.240,-575.861,-1.329.499,-1.495.323,0.0,394.315,8.128.872
3,31/03/2010,63.323.783,-12.911.715,50.412.073,-31.101.667,19.310.399,-2.072.394,-1.829.000,NaN,0,...,0.0,0.0,0.0,10.736.777,-3.386.217,446.287,0,0.0,-70.573,7.726.274
4,30/06/2010,66.884.448,-13.253.071,53.631.382,-34.244.725,19.386.655,-2.276.030,-1.897.404,NaN,0,...,0.0,0.0,0.0,11.441.831,-1.563.613,-1.541.420,0,0.0,-41.808,8.294.990


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 25 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   Data                                         60 non-null     object 
 1   Receita Bruta de Vendas e/ou Serviços        6 non-null      object 
 2   Deduções da Receita Bruta                    6 non-null      object 
 3   Receita Líquida de Vendas e/ou Serviços      60 non-null     object 
 4   Custo de Bens e/ou Serviços Vendidos         60 non-null     object 
 5   Resultado Bruto                              60 non-null     object 
 6   Despesas Com Vendas                          60 non-null     object 
 7   Despesas Gerais e Administrativas            60 non-null     object 
 8   Perdas pela Não Recuperabilidade de Ativos   53 non-null     float64
 9   Outras Receitas Operacionais                 60 non-null     int64  
 10  Outr

None

In [4]:
# Exibir o número de linhas e colunas
print(f"Demostrativo do Resultado - Linhas: {df_dem_resultado.shape[0]}, Colunas: {df_dem_resultado.shape[1]}")

Demostrativo do Resultado - Linhas: 60, Colunas: 25


In [6]:
# Contar os valores nulos em cada coluna do Balanço Patrimonial
nulos_demos_resul = df_dem_resultado.isnull().sum()
print("Valores nulos por coluna no demostrativo:")
display(nulos_demos_resul)

Valores nulos por coluna no demostrativo:


,0
Data,0
Receita Bruta de Vendas e/ou Serviços,54
Deduções da Receita Bruta,54
Receita Líquida de Vendas e/ou Serviços,0
Custo de Bens e/ou Serviços Vendidos,0
Resultado Bruto,0
Despesas Com Vendas,0
Despesas Gerais e Administrativas,0
Perdas pela Não Recuperabilidade de Ativos,7
Outras Receitas Operacionais,0


In [7]:
# Substituir valores nulos por zero no DataFrame df_dem_resultado
df_dem_resultado.fillna(0, inplace=True)

# Verificar se os valores nulos foram substituídos
print("DataFrame após substituir valores nulos por zero:")
display(df_dem_resultado.head())

# Se ainda quiser contar valores nulos, pode verificar agora:
nulos = df_dem_resultado.isnull().sum()
print("\nContagem de valores nulos (deve ser zero após a substituição):")
display(nulos)

DataFrame após substituir valores nulos por zero:


,Data,Receita Bruta de Vendas e/ou Serviços,Deduções da Receita Bruta,Receita Líquida de Vendas e/ou Serviços,Custo de Bens e/ou Serviços Vendidos,Resultado Bruto,Despesas Com Vendas,Despesas Gerais e Administrativas,Perdas pela Não Recuperabilidade de Ativos,Outras Receitas Operacionais,...,Resultado Não Operacional,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período
0,30/06/2009,55.891.489,-11.287.491,44.603.994,-24.613.196,19.990.798,-1.745.850,-1.834.082,0.0,0,...,0.0,0.0,0.0,11.807.475,-3.852.421,1.656.294,0,0.0,-1.876.852,7.734.496
1,30/09/2009,60.264.415,-12.386.796,47.877.620,-29.015.413,18.862.207,-1.756.580,-1.965.524,0.0,0,...,0.0,0.0,0.0,11.263.481,-2.354.096,-679.049,0,0.0,-927.768,7.302.568
2,31/12/2009,60.772.676,-13.139.804,47.632.871,-29.628.371,18.004.513,-1.785.407,-1.857.617,0.0,0,...,0.0,0.0,0.0,11.135.240,-575.861,-1.329.499,-1.495.323,0.0,394.315,8.128.872
3,31/03/2010,63.323.783,-12.911.715,50.412.073,-31.101.667,19.310.399,-2.072.394,-1.829.000,0.0,0,...,0.0,0.0,0.0,10.736.777,-3.386.217,446.287,0,0.0,-70.573,7.726.274
4,30/06/2010,66.884.448,-13.253.071,53.631.382,-34.244.725,19.386.655,-2.276.030,-1.897.404,0.0,0,...,0.0,0.0,0.0,11.441.831,-1.563.613,-1.541.420,0,0.0,-41.808,8.294.990



Contagem de valores nulos (deve ser zero após a substituição):


,0
Data,0
Receita Bruta de Vendas e/ou Serviços,0
Deduções da Receita Bruta,0
Receita Líquida de Vendas e/ou Serviços,0
Custo de Bens e/ou Serviços Vendidos,0
Resultado Bruto,0
Despesas Com Vendas,0
Despesas Gerais e Administrativas,0
Perdas pela Não Recuperabilidade de Ativos,0
Outras Receitas Operacionais,0


In [13]:
print("\após substituir valores nulos por zero:")
display(df_dem_resultado.head())

pós substituir valores nulos por zero:


,Data,Receita Bruta de Vendas e/ou Serviços,Deduções da Receita Bruta,Receita Líquida de Vendas e/ou Serviços,Custo de Bens e/ou Serviços Vendidos,Resultado Bruto,Despesas Com Vendas,Despesas Gerais e Administrativas,Perdas pela Não Recuperabilidade de Ativos,Outras Receitas Operacionais,...,Resultado Não Operacional,Receitas,Despesas,Resultado Antes Tributação/Participações,Provisão para IR e Contribuição Social,IR Diferido,Participações/Contribuições Estatutárias,Reversão dos Juros sobre Capital Próprio,Part. de Acionistas Não Controladores,Lucro/Prejuízo do Período
0,30/06/2009,55.891.489,-11.287.491,44.603.994,-24.613.196,19.990.798,-1.745.850,-1.834.082,0.0,0,...,0.0,0.0,0.0,11.807.475,-3.852.421,1.656.294,0,0.0,-1.876.852,7.734.496
1,30/09/2009,60.264.415,-12.386.796,47.877.620,-29.015.413,18.862.207,-1.756.580,-1.965.524,0.0,0,...,0.0,0.0,0.0,11.263.481,-2.354.096,-679.049,0,0.0,-927.768,7.302.568
2,31/12/2009,60.772.676,-13.139.804,47.632.871,-29.628.371,18.004.513,-1.785.407,-1.857.617,0.0,0,...,0.0,0.0,0.0,11.135.240,-575.861,-1.329.499,-1.495.323,0.0,394.315,8.128.872
3,31/03/2010,63.323.783,-12.911.715,50.412.073,-31.101.667,19.310.399,-2.072.394,-1.829.000,0.0,0,...,0.0,0.0,0.0,10.736.777,-3.386.217,446.287,0,0.0,-70.573,7.726.274
4,30/06/2010,66.884.448,-13.253.071,53.631.382,-34.244.725,19.386.655,-2.276.030,-1.897.404,0.0,0,...,0.0,0.0,0.0,11.441.831,-1.563.613,-1.541.420,0,0.0,-41.808,8.294.990


In [10]:
def verificar_colunas_duplicadas(df_dem_resultado):
    duplicatas = df_dem_resultado.columns[df_dem_resultado.columns.duplicated()]

    if duplicatas.any():
        print(f"Colunas duplicadas encontradas: {list(duplicatas)}")
    else:
        print("Não há colunas duplicadas.")

verificar_colunas_duplicadas(df_dem_resultado)


Não há colunas duplicadas.


In [ ]:
# Função para fazer upload de um arquivo para o bucket GCS em uma pasta específica
def upload_to_gcs(bucket_name, file_path, folder_name='dados_tratados'):
    bucket = client.get_bucket(bucket_name)
    file_name = os.path.basename(file_path)
    blob = bucket.blob(f'{folder_name}/{file_name}')
    blob.upload_from_filename(file_path)
    print(f'Arquivo {file_path} carregado para {bucket_name}/{folder_name}/{file_name}')

# Salvar o DataFrame tratado df_dem_resultado como um arquivo CSV localmente
csv_dem_resultado_path = '/content/dem_resultado_tratado.csv'
df_dem_resultado.to_csv(csv_dem_resultado_path, index=False)

# Fazer upload do arquivo CSV df_dem_resultado tratado para o Google Cloud Storage na pasta 'dados_tratados'
upload_to_gcs(bucket_name, csv_dem_resultado_path, folder_name='dados_tratados')

In [14]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

# Definir as credenciais e informações da conexão
servidor = '34.139.197.241'
basededados = 'Petrobras'
usuario = 'Wermelinger,V'
senha = '281431'

# Função para conectar ao banco de dados
def conectar_mysql(servidor, basededados, usuario, senha, use_database=True):
    try:
        conn = mysql.connector.connect(
            host=servidor,
            user=usuario,
            password=senha,
            database=basededados if use_database else None  # Se use_database for True, conecta diretamente ao banco
        )
        if conn.is_connected():
            print(f"Conexão bem-sucedida ao MySQL {'com banco de dados' if use_database else 'sem banco de dados'}")
            return conn
    except Error as e:
        print(f"Erro ao conectar ao MySQL: {e}")
        return None

# Conectar sem usar o banco de dados para criar o banco
conexao = conectar_mysql(servidor, basededados, usuario, senha, use_database=False)

# Função para criar o banco de dados
def criar_banco(conexao, basededados):
    try:
        cursor = conexao.cursor()
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {basededados}")
        conexao.commit()
        print(f"Banco de dados {basededados} criado ou já existente.")
    except Error as e:
        print(f"Erro ao criar banco de dados: {e}")

criar_banco(conexao, basededados)

# Fechar a conexão inicial
if conexao.is_connected():
    conexao.close()

# Conectar novamente, agora usando o banco de dados criado
conexao = conectar_mysql(servidor, basededados, usuario, senha)


Conexão bem-sucedida ao MySQL sem banco de dados
Banco de dados Petrobras criado ou já existente.
Conexão bem-sucedida ao MySQL com banco de dados


In [17]:
# Função para criar a tabela com base no DataFrame
def criar_tabela(conexao, df, nome_tabela):
    cursor = conexao.cursor()
    columns = ', '.join([f"`{col}` TEXT" for col in df.columns])

    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS `{nome_tabela}` (
        id INT AUTO_INCREMENT PRIMARY KEY,
        {columns}
    )
    """

    try:
        cursor.execute(create_table_query)
        conexao.commit()
        print(f"Tabela '{nome_tabela}' criada com sucesso.")
    except Error as e:
        print(f"Erro ao criar tabela: {e}")

# Exemplo de uso:
nome_tabela = 'dem_resultado'
criar_tabela(conexao, df_dem_resultado, nome_tabela)

Tabela 'dem_resultado' criada com sucesso.


In [18]:
# Função para consultar dados da tabela
def consultar_dados(conexao, nome_tabela):
    try:
        query = f"SELECT * FROM `{nome_tabela}`"
        df_resultado = pd.read_sql(query, con=conexao)
        print(df_resultado)
    except Error as e:
        print(f"Erro ao consultar dados: {e}")

# Exemplo de uso:
consultar_dados(conexao, nome_tabela)


Empty DataFrame
Columns: [id, Data, Receita Bruta de Vendas e/ou Serviços, Deduções da Receita Bruta, Receita Líquida de Vendas e/ou Serviços, Custo de Bens e/ou Serviços Vendidos, Resultado Bruto, Despesas Com Vendas, Despesas Gerais e Administrativas, Perdas pela Não Recuperabilidade de Ativos , Outras Receitas Operacionais, Outras Despesas Operacionais, Resultado da Equivalência Patrimonial, Financeiras, Receitas Financeiras, Despesas Financeiras, Resultado Não Operacional, Receitas, Despesas, Resultado Antes Tributação/Participações, Provisão para IR e Contribuição Social, IR Diferido, Participações/Contribuições Estatutárias, Reversão dos Juros sobre Capital Próprio, Part. de Acionistas Não Controladores, Lucro/Prejuízo do Período]
Index: []

[0 rows x 26 columns]


<ipython-input-18-2d8729a0cf3a>:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(query, con=conexao)


In [19]:
# Fechar a conexão
if conexao.is_connected():
    conexao.close()
    print("Conexão fechada.")


Conexão fechada.
